# Local Search
## Solving N-queen problem with min-conflict algorithm

* First generate a complete assignment for all variables (this set of assignments may conflict)

* Repeat the following steps until there are no conflicts:

    * Randomly Select a variable that causes conflicts
    
    * Reassign the value of this variable to another value that with the least constraint onflicts with other variables

In [7]:
import numpy as np
import time


def my_range(start, end):
    if start <= end:
        return range(start, end + 1)
    else:
        return range(start, end - 1, -1)


class Problem:
    char_mapping = ('·', 'Q')

    def __init__(self, n=4):
        self.n = n

    def is_valid(self, state):
        """
        check the state satisfy condition or not.
        :param state: list of points (in (row id, col id) tuple form)
        :return: bool value of valid or not
        """
        board = self.get_board(state)
        res = True
        for point in state:
            i, j = point
            condition1 = board[:, j].sum() <= 1
            condition2 = board[i, :].sum() <= 1
            condition3 = self.pos_slant_condition(board, point)
            condition4 = self.neg_slant_condition(board, point)
            res = res and condition1 and condition2 and condition3 and condition4
            if not res:
                break
        return res

    def is_satisfy(self, state):
        return self.is_valid(state) and len(state) == self.n

    def next_action(self, point):
        i, j = point
        if 0 <= i < self.n and 0 <= j < self.n and i * self.n + j < self.n ** 2 - 1:
            j += 1
            if j == self.n:
                j = 0
                i += 1
            return i, j
        else:
            return None

    def pos_slant_condition(self, board, point):
        i, j = point
        tmp = min(self.n - i - 1, j)
        start = (i + tmp, j - tmp)
        tmp = min(i, self.n - j - 1)
        end = (i - tmp,  j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def neg_slant_condition(self, board, point):
        i, j = point
        tmp = min(i, j)
        start = (i - tmp, j - tmp)
        tmp = min(self.n - i - 1, self.n - j - 1)
        end = (i + tmp, j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def get_board(self, state):
        board = np.zeros([self.n, self.n], dtype=int)
        for point in state:
            board[point] = 1
        return board

    def print_state(self, state):
        board = self.get_board(state)
        print('_' * (2 * self.n + 1))
        for row in board:
            for item in row:
                print(f'|{Problem.char_mapping[item]}', end='')
            print('|')
        print('-' * (2 * self.n + 1))

In [8]:
def min_conflict(problem):
    import random
    action_stack = []
    N = problem.n
    col = np.zeros(N, dtype=int)
    lob = np.zeros(2*N-1, dtype=int)
    rob = np.zeros(2*N-1, dtype=int)
    for i in range(N):
        c = random.randint(0, N-1)
        action_stack.append((i, c))
        col[c] += 1
        lob[i+c] += 1
        rob[N-1-i+c] += 1
    while not problem.is_satisfy(action_stack):
        cof_list = []
        for i in range(N):
            pos = action_stack[i]
            r = pos[0]; c = pos[1]
            if col[c] > 1 or lob[r+c] > 1 or rob[N-1-r+c] > 1:
                cof_list.append(i)
        i = random.choice(cof_list)
        pos = action_stack[i]
        r = pos[0]; c = pos[1]
        col[c] -= 1
        lob[r+c] -= 1
        rob[N-1-r+c] -= 1
        id = -1
        minp = 3*N
        for j in range(0, N):
            p = col[j]+lob[r+j]+rob[N-1-r+j]
            if minp >= p:
                minp = p
                if id == -1 or minp > p:
                    id = j
                elif minp == p and random.randint(0, 1) == 1:
                    id = j
        if id != -1:
            action_stack[i] = (r, id)
        col[id] += 1
        lob[r+id] += 1
        rob[N-1-r+id] += 1
        yield action_stack

In [9]:
# test block
n = 100
render = (n == 15)
p = Problem(n)
if render:
    import pygame

    w, h = 90 * n + 10, 90 * n + 10
    screen = pygame.display.set_mode((w, h))
    screen.fill('white')
    action_generator = min_conflict(p)
    clk = pygame.time.Clock()
    queen_img = pygame.image.load('./queen_s.png')
    queen_img = pygame.transform.scale(queen_img, (80, 80))
    while True:
        for event in pygame.event.get():
            if event == pygame.QUIT:
                exit()
        try:
            actions = next(action_generator)
            screen.fill('white')
            for i in range(n + 1):
                pygame.draw.rect(screen, 'black', (i * 90, 0, 10, h))
                pygame.draw.rect(screen, 'black', (0, i * 90, w, 10))
            for action in actions:
                i, j = action
                screen.blit(queen_img, (10 + 90 * j, 10 + 90 * i))
            pygame.display.flip()
        except StopIteration:
            pass
        clk.tick(5)
    pass
else:
    start_time = time.time()
    for actions in min_conflict(p):
        pass
    print(time.time() - start_time)
    p.print_state(actions)

0.06401276588439941
_____________________
|·|·|·|·|·|·|·|Q|·|·|
|Q|·|·|·|·|·|·|·|·|·|
|·|·|·|·|Q|·|·|·|·|·|
|·|·|·|·|·|·|·|·|Q|·|
|·|Q|·|·|·|·|·|·|·|·|
|·|·|·|Q|·|·|·|·|·|·|
|·|·|·|·|·|·|Q|·|·|·|
|·|·|Q|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|Q|
|·|·|·|·|·|Q|·|·|·|·|
---------------------
